In [19]:
import pandas as pd

# Ezzel ne foglalkozz, adat betoltes
def load_and_process_measurements(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    df['label'] = -1
    current_label = -1
    in_measurement = False

    for idx, row in df.iterrows():
        if (row == 'X').any():
            current_label += 1
            in_measurement = True
            df.at[idx, 'label'] = current_label
        elif (row == 'O').any():
            df.at[idx, 'label'] = current_label
            in_measurement = False
        elif in_measurement:
            df.at[idx, 'label'] = current_label
        else:
            df.at[idx, 'label'] = -1

    df = df[~df.apply(
        lambda row: row.astype(str).eq('X').any() or row.astype(str).eq('O').any(), axis=1)]

    df = df[df['label'] != -1]
    df.reset_index(inplace=True, drop=True)

    df = df.astype(float)

    df['time'] = df['Timestamp'] - df['Timestamp'].iloc[0]

    df['relative_time'] = df.groupby('label')['Timestamp'].transform(lambda x: x - x.iloc[0])

    df = df.drop(columns=['Timestamp'])

    return df


In [20]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ha ezen valtoztatsz az egesz el fog baszodni(hacsak mindenhol nem igazitod hozza) es a modellt is ujra kell trainelni.
# De am csak a datat dolgozza fel
def prepare_sequence_data(df: pd.DataFrame, max_len: int = None):
    feature_cols = df.columns.difference(['label', 'time'])
    sequences = []
    labels = []
    
    grouped = df.groupby('label')

    for _, group in grouped:
        seq = group[feature_cols].values
        sequences.append(seq)

    if max_len is None:
        max_len = max(len(seq) for seq in sequences)

    sequences_padded = pad_sequences(sequences, maxlen=max_len, padding='post', dtype='float32')
    labels = np.array(labels)

    return sequences_padded

In [21]:
# Itt írd át a pathet arra a .csv fajlra ahol vannak a measurementjeid. Nagyon fontos hogy a .csv fajl ugyanugy van generalva mint amit adtal,
# es egyik mérés se lehet tobb soros mint a correct.csv leghosszabb merese(65 sor).
path = r'incorrect_circles.csv'
# Na ez a measurement number azt tudja hogy ha beadod neki a csvt amiben benne vannak a measurementek(minimum 1 de lehet tobb is) akk ugye 
# a preprocesselos resz beszamozza a measurementeket 0 tol kezdve(tehat az elso Xes sortol az elso Oos sorig van a 0. measurement) es ez a szam hatarozza meg
# hogy epp hanyadik measurementrol akarod eldonteni hogy kor vagy nem. A measurement number nem lehet nagyobb
# mint ahany meresed van a betoltott csvben.(sztem logikus)
measurement_number = 1

# Valszeg jo de írd át ha szar/masik mappaba akarod tenni
model_path = r'95test_acc_v1.h5'
# Ha átírod új model kell
max_rows = 65

new_data = load_and_process_measurements(path)
print('Correct circles length:', new_data.shape[0], 'Number of measurements:', new_data['label'].nunique(), 'Avg measurement length:', new_data.shape[0] / new_data['label'].nunique())
print('Longest measuremet:', new_data['label'].value_counts().max(), 'Shortest:', new_data['label'].value_counts().min())
print('measurement_number in range of data length:', measurement_number < new_data['label'].nunique())
if measurement_number >= new_data['label'].nunique(): 
    raise ValueError("Figyeljel ocskos")
new_data.head()

Correct circles length: 1036 Number of measurements: 27 Avg measurement length: 38.370370370370374
Longest measuremet: 49 Shortest: 28
measurement_number in range of data length: True


,linear_accelerationX,linear_accelerationY,linear_accelerationZ,magX,magY,magZ,gyroX,gyroY,gyroZ,accX,accY,accZ,label,time,relative_time
0,0.107210,0.308428,0.074314,22.900391,-21.643066,-77.124023,-1.770020,7.385254,-3.051758,-0.069824,0.828369,0.593018,0.0,0.000000,0.000000
1,0.127767,0.283521,0.009217,23.742676,-21.862793,-75.769043,3.601074,1.708984,-3.173828,-0.047119,0.808228,0.528076,0.0,0.002003,0.002003
2,0.049542,0.193477,0.018429,23.059082,-20.874023,-76.477051,2.014160,-6.469727,2.990723,-0.124512,0.721436,0.537598,0.0,0.003003,0.003003
3,0.136362,0.284401,0.031450,23.144531,-21.557617,-75.585938,8.850098,-1.098633,1.525879,-0.035400,0.817139,0.551147,0.0,0.060880,0.060880
4,0.120422,0.244764,0.148342,22.680664,-21.618652,-75.476074,7.507324,0.488281,-2.807617,-0.049316,0.781616,0.670532,0.0,0.120578,0.120578


In [22]:
prepared_data = prepare_sequence_data(new_data, max_len=max_rows)

In [23]:
# Ehhez mar lusta voltam szval szar chatgpt kod de mukodnie kene
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model(model_path)

# Assume you have your processed data loaded:
# X_correct, y_correct = prepare_sequence_data(correct, label_value=1)

# Pick one measurement (e.g., the first one)
example = prepared_data[measurement_number]  # shape: (timesteps, features)

# Reshape to match model input: (1, timesteps, features)
example_input = np.expand_dims(example, axis=0)

# Predict
prediction = model.predict(example_input)[0][0]

# Print result
print(f"Predicted probability of being correct: {prediction:.4f}")
print("Predicted label:", "Correct" if prediction >= 0.5 else "Incorrect")


1/1 [==============================] - 0s 53ms/step
Predicted probability of being correct: 0.0009
Predicted label: Incorrect
